# Palmer Penguins Clustering

## Dataset treatment

In [30]:
import pandas as pd
from palmerpenguins import load_penguins
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler, MinMaxScaler

from kmeans import apply_kmeans


In [40]:
df = load_penguins()
df = df.dropna().reset_index(drop=True)
df

,species,island,bill_length_mm,bill_depth_mm,flipper_length_mm,body_mass_g,sex,year
0,Adelie,Torgersen,39.1,18.7,181.0,3750.0,male,2007
1,Adelie,Torgersen,39.5,17.4,186.0,3800.0,female,2007
2,Adelie,Torgersen,40.3,18.0,195.0,3250.0,female,2007
3,Adelie,Torgersen,36.7,19.3,193.0,3450.0,female,2007
4,Adelie,Torgersen,39.3,20.6,190.0,3650.0,male,2007
...,...,...,...,...,...,...,...,...
328,Chinstrap,Dream,55.8,19.8,207.0,4000.0,male,2009
329,Chinstrap,Dream,43.5,18.1,202.0,3400.0,female,2009
330,Chinstrap,Dream,49.6,18.2,193.0,3775.0,male,2009
331,Chinstrap,Dream,50.8,19.0,210.0,4100.0,male,2009


In [41]:
df = pd.get_dummies(df, columns=['island', 'sex'], drop_first=True)
df

,species,bill_length_mm,bill_depth_mm,flipper_length_mm,body_mass_g,year,island_Dream,island_Torgersen,sex_male
0,Adelie,39.1,18.7,181.0,3750.0,2007,False,True,True
1,Adelie,39.5,17.4,186.0,3800.0,2007,False,True,False
2,Adelie,40.3,18.0,195.0,3250.0,2007,False,True,False
3,Adelie,36.7,19.3,193.0,3450.0,2007,False,True,False
4,Adelie,39.3,20.6,190.0,3650.0,2007,False,True,True
...,...,...,...,...,...,...,...,...,...
328,Chinstrap,55.8,19.8,207.0,4000.0,2009,True,False,True
329,Chinstrap,43.5,18.1,202.0,3400.0,2009,True,False,False
330,Chinstrap,49.6,18.2,193.0,3775.0,2009,True,False,True
331,Chinstrap,50.8,19.0,210.0,4100.0,2009,True,False,True


In [42]:
X = df.drop('species', axis=1)
y = df['species']

scaler = MinMaxScaler()
X_scaled = scaler.fit_transform(X)

df_standardized = pd.DataFrame(X_scaled, columns=X.columns)
df_standardized['class'] = y

df_standardized

,bill_length_mm,bill_depth_mm,flipper_length_mm,body_mass_g,year,island_Dream,island_Torgersen,sex_male,class
0,0.254545,0.666667,0.152542,0.291667,0.0,0.0,1.0,1.0,Adelie
1,0.269091,0.511905,0.237288,0.305556,0.0,0.0,1.0,0.0,Adelie
2,0.298182,0.583333,0.389831,0.152778,0.0,0.0,1.0,0.0,Adelie
3,0.167273,0.738095,0.355932,0.208333,0.0,0.0,1.0,0.0,Adelie
4,0.261818,0.892857,0.305085,0.263889,0.0,0.0,1.0,1.0,Adelie
...,...,...,...,...,...,...,...,...,...
328,0.861818,0.797619,0.593220,0.361111,1.0,1.0,0.0,1.0,Chinstrap
329,0.414545,0.595238,0.508475,0.194444,1.0,1.0,0.0,0.0,Chinstrap
330,0.636364,0.607143,0.355932,0.298611,1.0,1.0,0.0,1.0,Chinstrap
331,0.680000,0.702381,0.644068,0.388889,1.0,1.0,0.0,1.0,Chinstrap


## Clustering

In [44]:
from kmeans import apply_kmeans

In [47]:
df["Cluster_KMeans"] = apply_kmeans(X, 3)
df

,species,bill_length_mm,bill_depth_mm,flipper_length_mm,body_mass_g,year,island_Dream,island_Torgersen,sex_male,Cluster_KMeans
0,Adelie,39.1,18.7,181.0,3750.0,2007,False,True,True,1
1,Adelie,39.5,17.4,186.0,3800.0,2007,False,True,False,1
2,Adelie,40.3,18.0,195.0,3250.0,2007,False,True,False,1
3,Adelie,36.7,19.3,193.0,3450.0,2007,False,True,False,1
4,Adelie,39.3,20.6,190.0,3650.0,2007,False,True,True,1
...,...,...,...,...,...,...,...,...,...,...
328,Chinstrap,55.8,19.8,207.0,4000.0,2009,True,False,True,0
329,Chinstrap,43.5,18.1,202.0,3400.0,2009,True,False,False,1
330,Chinstrap,49.6,18.2,193.0,3775.0,2009,True,False,True,1
331,Chinstrap,50.8,19.0,210.0,4100.0,2009,True,False,True,0
